In [2]:
!pip install openai

In [111]:
import os
from openai import OpenAI
from pathlib import Path
import time
import ast

api_key = "" 
client = OpenAI(api_key=api_key)


In [113]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV,train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
import seaborn as sns
from tqdm import tqdm
from sklearn.utils import Bunch
import joblib

In [115]:
def generate_quiz(topic, max_tokens = 200):
    prompt = f"Create 3 question MCQ quiz on the following question: {topic},do not add anything other than the questions"
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        n=1,
        max_tokens = max_tokens,
    )
    #print("Tokens used:", response.usage.total_tokens)
    return response.choices[0].message.content
    

def quiz_answers(prompt, max_tokens = 200):
    prompt = f"""
    Look at the following quiz and carefully determine the correct answers based on the information provided. For each question:
    1. Read the question and all options carefully.
    2. Eliminate incorrect options.
    3. Choose the correct answer based on reasoning or calculation.
    
    {quiz}
    
    After reasoning through all questions, provide ONLY the answers in this format: A,B,C,D].
    Ensure all answers are correct.
    """
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        n=1,
        max_tokens = max_tokens,
        temperature=0.7,
    )
    #print("Tokens used:", response.usage.total_tokens)
    return response.choices[0].message.content

def grade_quiz(quiz_answer, user_answer,max_tokens = 200):
    prompt = f"""
    Compare the user's answers with the correct answers
    **User's answers**: {user_answer}
    **Correct answers**: {quiz_answer}

    Each answer is seperated by a ','
    **Only return the final score** as a percentage (e.g., 75.00%).
    Do not include any explanations, text, or additional information.
    
    Return the percentage score formatted to two decimal places, like this: 80.00%.
    """
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        n=1,
        temperature=0,
        max_tokens = max_tokens,
    )
    #print("Tokens used:", response.usage.total_tokens)
    return response.choices[0].message.content

def assign_skill_level(quiz_score):
  if quiz_score >= 90:
    return "Advanced"
  elif quiz_score >= 70:
    return "Intermediate"
  else:
    return "Beginner"

def adjust_quiz_difficulty(quiz_score):
  if quiz_score >= 100:
    return "Much more difficult"
  elif quiz_score >= 90:
      return "More difficult"
  elif quiz_score >= 70:
    return "Slightly more difficult"
  elif quiz_score >= 50:
      return "Similar"
  else:
    return "Slightly easier"

def generate_additional_quiz(quiz, new_difficulty, topic,max_tokens=130):
    prompt = f"Using this quiz as the baseline \n {quiz} \n create a {new_difficulty} quiz on {topic}, \n(MCQ Only, do not give the answers)"
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens,
        n=1
    )
    #print("Tokens used:", response.usage.total_tokens)
    return response.choices[0].message.content
    

def explain_answers(quiz, quiz_answers, user_answer, max_tokens = 150):
    prompt = f"Explain any question that the user got wrong in this quiz \n {quiz} \n these are the answers \n {quiz_answers} and this is the user's answers \n {user_answer} \n explain how to get the answers \n ONLY explain incorrect answers"
    response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role":"user","content":prompt}],
    n=1,
    max_tokens = max_tokens,
    temperature = 0.5,
    )
    return response.choices[0].message.content

def respond_to_query(query, max_tokens=100):
    prompt = f"{query}"
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        max_tokens = max_tokens,
        n=1
    )
    #print("Tokens used:", response.usage.total_tokens)
    return response.choices[0].message.content

def get_question_difficulty(quiz ,max_tokens = 200):
    prompt = f"Rate the difficulty of each question in this quiz out of 10 (1 decimal point allowed) \n{quiz} \n , your response should ONLY contain the difficulty in this format [0.2,1,5,8,10]"
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature = 0,
        max_tokens = max_tokens,
        n=1
    )
    #print("Tokens used:", response.usage.total_tokens)
    return response.choices[0].message.content

def get_topic_difficulty(topic,  max_tokens = 200):
    prompt = f"Rate the difficulty of this topic: {topic}, 10 being the most difficult (1 decimal point), your response should ONLY contain the difficulty"
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature = 0,
        n=1,
        max_tokens = max_tokens,
    )
    #print("Tokens used:", response.usage.total_tokens)
    return response.choices[0].message.content

In [117]:
topic = input("What topic would you like to study?: ")
quiz = generate_quiz(topic)
print(quiz)

What topic would you like to study?:  Addition


1. What is the sum of 8 and 5?  
   a) 12  
   b) 13  
   c) 14  

2. Calculate the total of 15 and 9.  
   a) 23  
   b) 24  
   c) 25  

3. What is the result of adding 7 and 6?  
   a) 12  
   b) 13  
   c) 14  


In [27]:
start_time = time.time()
user_answer = input("Please Enter your quiz answers")
end_time = time.time()
time_taken = end_time - start_time
print(f"Time taken to respond: {time_taken:.2f} seconds")

Please Enter your quiz answers b, c, d


Time taken to respond: 20.66 seconds


In [29]:
quiz_answer = quiz_answers(quiz)
print(quiz_answer)

b,b,d


In [31]:
response = grade_quiz(quiz_answer,user_answer)
print(response)
quiz_score = float(response.strip('%'))
new_difficulty = adjust_quiz_difficulty(quiz_score)
print(new_difficulty)

66.67%
Similar


In [33]:
question_difficulty = get_question_difficulty(quiz)
print("Question_Difficulty: ",question_difficulty)
question_difficulty = ast.literal_eval(question_difficulty)
average_question_difficulty = sum(question_difficulty) / len(question_difficulty)
print("Average Question Difficulty:" ,average_question_difficulty)
topic_difficulty = get_topic_difficulty(topic)
topic_difficulty = float(topic_difficulty)
print("Topic Difficulty: ",topic_difficulty)
average_time_taken = time_taken/3
average_time_taken = round(average_time_taken,4)
print("Average Time Taken: ",average_time_taken)

Question_Difficulty:  [2.0, 2.0, 1.5]
Average Question Difficulty: 1.8333333333333333
Topic Difficulty:  3.5
Average Time Taken:  6.8878


In [59]:
additional_quiz = generate_additional_quiz(quiz, new_difficulty,topic)
print(additional_quiz)

1. What is 5 + 2?
A) 6
B) 7
C) 8
D) 9
2. What is 6 + 1?
A) 6
B) 7
C) 8
D) 9
3. What is 4 + 3?
A) 6
B) 7
C) 8
D) 9


In [533]:
print(quiz_answer)
topic_difficulty = get_topic_difficulty(topic)
print(topic_difficulty)

[B, A, B]
3


In [100]:
data = pd.read_csv('modified_file.csv')
X = data[['Time Taken (QNS/Seconds)', 'Question Difficulty', 'Topic Difficulty', 'Score']]
Y = data['Skill_Level_encoded']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print("Training set size:", X_train.shape)
print("Testing set size:", X_test.shape)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = joblib.load('skill_level_model.joblib')
scaler = StandardScaler()
scaler.fit(X_train)

Training set size: (16000, 4)
Testing set size: (4000, 4)


StandardScaler()

In [108]:
new_data = pd.DataFrame({
    'Time Taken (QNS/Seconds)': [average_time_taken],
    'Question Difficulty': [average_question_difficulty],
    'Topic Difficulty': [topic_difficulty],
    'Score': [quiz_score]
})

new_data_scaled = scaler.transform(new_data)

# Predict with the trained model
predictions = model.predict(new_data_scaled)
label_mapping = {0: "Beginner", 1: "Intermediate", 2: "Advanced"}

# Decode predictions
decoded_predictions = [label_mapping[pred] for pred in predictions]

print("Predictions:", decoded_predictions)

Predictions: ['Beginner']
